In [5]:
import pandas as pd
import numpy as np

# 모델
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# test mse/ incentive estimate

In [23]:
def testMSEnIncentive(pred):
    realGens1030 = np.array([0, 0, 0, 0, 0, 0,
                            0, 4, 20, 56,
                            65, 79, 81, 75,
                            62, 45, 22, 3, 0,
                            0, 0, 0, 0, 0])    
    
    rss = np.power(pred - realGens1030, 2)
    rss1 = rss[6:10]
    rss2 = rss[10:14]
    rss3 = rss[14:19]
    mse1 = sum(rss1/len(rss1))
    mse2 = sum(rss2/len(rss2))
    mse3 = sum(rss3/len(rss3))
    mse = [mse1,mse2,mse3]

    score_df = abs(pred - realGens1030)*100/99
    score = []
    for idx, i in enumerate(score_df):
        if i <=6:
            score.append(realGens1030[idx] * 4)
        elif (i <=8) & (i > 6):
            score.append(realGens1030[idx] * 3)
        else:
            score.append(0)
    score1 = score[6:10]
    score2 = score[10:14]
    score3 = score[14:19]
    final_score = [score1,score2,score3]

    return mse, final_score

# spliting test data set

In [6]:
## test data set 만드는 함수
def make_test_xy(wf,gf):
    gf.columns = ["time","m0","m1","m2","m3","m4"]
    train = pd.merge(gf,wf,on="time")
    train["time"] = pd.to_datetime(train["time"])
    train["month"]=train["time"].dt.month
    train["day"]=train["time"].dt.day
    train["hour"]=train["time"].dt.hour
    
    train_1 = train[(train["hour"]<=10) & (train["hour"]>=7)]
    train_2 = train[(train["hour"]<=14) & (train["hour"]>=11)]
    train_3 = train[(train["hour"]<=19) & (train["hour"]>=15)]
    
    train11 = train_1[["m0","m1","m2","m3","uv_idx","elevation"]]
    train12 = train_2[["m0","m1","m2","m3","uv_idx","elevation"]]
    train13 = train_3[["m0","m1","m2","m3","uv_idx","elevation"]]
    train21 = train_1.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train22 = train_2.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train23 = train_3.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    
    return train11,train12,train13,train21,train22,train23

# 데이터 load

In [11]:
import joblib

In [67]:
# 예측에 사용할 데이터 셋 불러오기 
x_pred_ori = pd.read_csv("../data/gen_fcst_17_2023-10-30.csv")
x_pred_wf = pd.read_csv("../data/weather_forecasts/wf17_2023-10-30.csv")

In [68]:
# 함수에 넣어서 데이터셋 분리
test11,test12,test13,test21,test22,test23 = make_test_xy(x_pred_wf,x_pred_ori)

In [69]:
model11=joblib.load("../model/시간1회귀fitting_season9-11_r2.pkl")
model12=joblib.load("../model/시간2회귀fitting_season9-11_r2.pkl")
model13=joblib.load("../model/시간3회귀fitting_season9-11_r2.pkl")

In [70]:
# 예측 진행 (1차 모델링 y_hat)
y_pred11=model11.predict(test11)
y_pred12=model12.predict(test12)
y_pred13=model13.predict(test13)

In [71]:
model21=joblib.load("../model/시간1잔차fitting_season9-11_r2.pkl")
model22=joblib.load("../model/시간2잔차fitting_season9-11_r2.pkl")
model23=joblib.load("../model/시간3잔차fitting_season9-11_r2.pkl")

In [72]:
y_pred21 = model21.predict(test21)
y_pred22 = model22.predict(test22)
y_pred23 = model23.predict(test23)

In [73]:
result1 = y_pred11+y_pred21
result2 = y_pred12+y_pred22
result3 = y_pred13+y_pred23
result = np.concatenate((result1,result2,result3))
result

array([ 0.73153268,  6.59974534, 26.63852524, 48.03316934, 63.03039959,
       72.09729688, 74.00698146, 65.88170122, 50.30159219, 35.53265698,
       14.17607442,  1.50903526,  0.75730511])

In [74]:
pred = [0,0,0,0,0,0,
    0.73153268,  6.59974534, 26.63852524, 48.03316934, 63.03039959,
       72.09729688, 74.00698146, 65.88170122, 50.30159219, 35.53265698,
       14.17607442,  1.50903526,  0.75730511,
    0,0,0,0,0]
len(pred)


24

In [75]:
testMSEnIncentive(pred)

([28.708556005498195, 45.893079270015775, 58.09872550286908],
 [[0, 16, 60, 0], [260, 237, 243, 0], [0, 0, 66, 12, 0]])

In [22]:
a=[1,2,3,4]
a[0:1]

[1]

In [ ]:
[0, 0, 0, 0, 0, 0,
                            0, 4, 20, 56,
                            65, 79, 81, 75,
                            62, 45, 22, 3, 0,
                            0, 0, 0, 0, 0]